In [1]:
%load_ext lab_black

import os
import time
from crewai import Agent, Task, Process, Crew
from crewai.task import TaskOutput
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

from pydantic import BaseModel

import warnings

load_dotenv()
warnings.filterwarnings("ignore", category=Warning)
warnings.filterwarnings("ignore", category=UserWarning)

C:\Users\caio_barros\AppData\Local\anaconda3\envs\ai-agents\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# ollama version

# OLLAMA_LLM_MODEL = "llama3.1:latest"
# BASE_URL = os.environ["OPENAI_API_BASE"]
# llm = ChatOpenAI(model=OLLAMA_LLM_MODEL, base_url=BASE_URL)

# Groq
GROQ_MODEL = "llama-3.1-8b-instant"
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
llm = ChatGroq(model=GROQ_MODEL, api_key=GROQ_API_KEY)

In [3]:
def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(
        f"""
        Task completed!
        Task: {output.description}
        Output: {output.raw_output}
    """
    )


class OutputFormat(BaseModel):
    user_profile: str


# Agent: Profile Analysis
profile_analysis_agent = Agent(
    role="Profile Analysis Agent",
    goal="With the client profile here: {user_profile}, your task is to analyze the client's profile, including health conditions, dietary preferences, and lifestyle, to understand their unique nutritional needs.",
    backstory="""
        This agent focuses on gathering and interpreting the client's health data, dietary restrictions, and lifestyle habits to provide a detailed understanding of their nutritional needs.
    """,
    llm=llm,
)

# Agent: Diet Recommendation
diet_recommendation_agent = Agent(
    role="Diet Recommendation Agent",
    goal="Generate a personalized and evidence-based nutrition plan tailored to the client's profile and health goals.",
    backstory="""
        This agent uses the client's profile analysis to create a balanced meal plan, considering their dietary preferences, health conditions, and goals.
    """,
    llm=llm,
)

# Agent: Educational Support
educational_support_agent = Agent(
    role="Educational Support Agent",
    goal="Provide ongoing education and resources on healthy eating habits, nutritional guidelines, and cooking techniques.",
    backstory="""
        This agent focuses on educating the client about nutrition, providing tips for grocery shopping, healthy cooking, and making informed dietary choices.
    """,
    llm=llm,
)

nutritionist_agent = Agent(
    role="Nutritionist Expert",
    goal="Provide comprehensive and personalized nutritional support to clients by focusing on three primary areas: understanding individual needs, developing tailored nutrition plans, and offering ongoing education",
    backstory="""
        This agent focuses on creating the schedule, reuniting all retrieved information and returning it to the user in a Markdown format.
    """,
    llm=llm,
)

# Task: Profile Analysis
profile_analysis_task = Task(
    description="Analyze the client's profile to understand their dietary needs, health conditions, and preferences.\n Client profile: {user_profile}",
    expected_output="A text string with the overview, dietary needs, summary of health condition, daily calories consumption, food preferences and restrictions, and lifestyle and activity level",
    agent=profile_analysis_agent,
)

# Task: Diet Recommendation
diet_recommendation_task = Task(
    description="Create a personalized nutrition plan based on the client's profile analysis, balancing among the carbohydrates, proteins, and fats",
    expected_output="""A documentation contained: Nutrition Plan, Personalized meal plan tailored to client's goals, Dietary recommendations based on health conditions, 
    - Weekly Meal Plan (3 options for each meal): Options for each day of the week and the time for each meal, containing the quantity for each portion (e.g. grams, mL, etc)
        **Breakfast:** Good options for the morning\n 
        **Lunch:** Balanced meals with portion sizes\n
        **Afternoon Snacks:** Healthy snack ideas\n
        **Dinner:** Recipes and cooking tips\n 
        **Supper:** Snack ideas before sleep\n
    """,
    agent=diet_recommendation_agent,
    context=[profile_analysis_task],
)

# Task: Educational Support
educational_support_task = Task(
    description="Provide educational resources and guidance on healthy eating habits and nutritional guidelines.",
    expected_output="A document list with Educational Resources, Nutritional guidelines, Tips for grocery shopping and Healthy cooking techniques. Respond in a direct language and formal tone.",
    agent=educational_support_agent,
    context=[profile_analysis_task],
)

nutritionist_planner_task = Task(
    description="Create a daily meal plan for the user, providing educational resources and guidance on healthy eating habits and nutritional guidelines. Preferably, use the Brazilian food/cuisine.",
    expected_output="A completed meal planner for the client, with all preferences and goals. Respond in a direct language and formal tone.",
    agent=educational_support_agent,
    output_format="markdown",
    output_format_model=OutputFormat,
    output_format_description=(
        "The output format is a markdown file to save in a pdf file with this structure:\n"
        "1. ## Overview;\n"
        "2. ## Daily plan, for each day and 3 options for each meal;\n"
            "2.1 - **### Breakfast:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
            "2.2 - **### Lunch:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
            "2.3 - **### Afternoon Snacks:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
            "2.4 - **### Dinner:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
            "2.5 - **### Supper:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
            "2.6 - **### Supplementation: List possible supplements to fit in the meal plan, like omega 3, caffeine, whey protein, magnesium, beta-alanine, etc**
        "3. ## Educational Resources and Guidance;\n"
        "4. ## Activities, tips, and exercises plan to the gym;\n"
            "e.g. \n"
            "4.1 - ### Day 1 (Monday): Back and biceps day and 30min cardio.,\n"
            "4.2 - ### Day 2 (Tuesday): Chest, triceps, and abs day exercises...etc\n"
            "4.3 - ...\n"
        "5. ## Cooking tips and how to cook some foods to this plan;\n"
        "6. ## Other important information to the user based on the profile;\n"
    ),
    context=[profile_analysis_task, diet_recommendation_task, educational_support_task],
    output_file="outputs/caiobarros_planner.pdf",
    create_directory=True,
)

In [3]:
class OutputFormat(BaseModel):
    user_profile: str


# Agent: Profile Analysis
profile_analysis_agent = Agent(
    role="Profile Analysis Agent",
    goal="""With the client's profile: {user_profile}\n, thoroughly analyze the client's profile, including health conditions, dietary preferences, and lifestyle,
    to accurately assess their unique nutritional needs and fitness goals.""",
    backstory="""
        This agent specializes in deep analysis of the client's health data, dietary restrictions, and lifestyle habits, 
        providing a detailed understanding of their nutritional needs. It aims to identify key areas for improvement and tailor recommendations accordingly.
    """,
    llm=llm,
)

# Agent: Diet Recommendation
diet_recommendation_agent = Agent(
    role="Diet Recommendation Agent",
    goal="Design a highly personalized and evidence-based nutrition plan that aligns with the client's unique profile and specific health and fitness objectives.",
    backstory="""
        This agent uses insights from the profile analysis to develop a balanced and tailored meal plan. 
        It ensures the plan is nutritionally sound, realistic, and sustainable, incorporating the client’s preferences, goals, and any necessary dietary adjustments.
    """,
    llm=llm,
)

# Agent: Educational Support
educational_support_agent = Agent(
    role="Educational Support Agent",
    goal="Provide comprehensive educational resources on healthy eating, nutritional science, and practical cooking techniques to empower the client in making informed dietary choices.",
    backstory="""
        This agent is dedicated to educating the client about nutrition, and offering practical tips for grocery shopping, meal preparation, and cooking. 
        It focuses on delivering actionable knowledge to help the client maintain a healthy lifestyle long-term.
    """,
    llm=llm,
)

nutritionist_agent = Agent(
    role="Nutritionist Expert",
    goal="Deliver a holistic and personalized nutritional plan that combines analysis, tailored meal planning, and educational resources to support the client's overall health and fitness journey.",
    backstory="""
        This agent synthesizes all the gathered information, creating a comprehensive nutrition and lifestyle plan in a clear and actionable format. 
        The goal is to ensure the client receives practical, easy-to-follow advice that aligns with their goals.
    """,
    llm=llm,
)

# Task: Profile Analysis
profile_analysis_task = Task(
    description="""Conduct a detailed analysis of the client's profile, focusing on their dietary needs, health conditions, 
    and lifestyle preferences to create an accurate baseline for personalized nutrition planning.\n Client profile: {user_profile}""",
    expected_output="""A concise and informative overview of the client's nutritional needs, health conditions, daily caloric requirements, 
    food preferences and restrictions, as well as their lifestyle and activity level. 
    This should highlight key areas for nutritional improvement.""",
    agent=profile_analysis_agent,
)

# Task: Diet Recommendation
diet_recommendation_task = Task(
    description="""Develop a personalized nutrition plan that balances carbohydrates, proteins, and fats, while incorporating salads and vegetables in lunch and dinner to meet 
    the client's health and fitness goals. Use the 'current_diet' in the client profile to create the new personalized plan, but not only it.""",
    expected_output="""A well-structured documentation that includes:
    - A comprehensive Nutrition Plan tailored to the client's goals and profile.
    - A Weekly Meal Plan offering 3 varied options for each meal (breakfast, lunch, afternoon snacks, dinner, supper), including portion sizes (grams, mL) and incorporating salads and vegetables in lunch and dinner.
    - Dietary recommendations based on the client's health conditions, ensuring all nutritional needs are met.
    """,
    agent=diet_recommendation_agent,
    context=[profile_analysis_task],
)

# Task: Educational Support
educational_support_task = Task(
    description="""Create a detailed educational guide that covers healthy eating habits, nutritional guidelines, and practical cooking techniques, 
    emphasizing the importance of adequate water intake (measured in liters), and any other tips based on the profile analysis.""",
    expected_output="""A comprehensive guide that includes: Educational Resources on nutrition, Detailed Nutritional Guidelines (including water intake in liters), 
    Tips for Grocery Shopping, and Practical Cooking Techniques. The content should be clear, actionable, and tailored to the client's needs.""",
    agent=educational_support_agent,
    context=[profile_analysis_task],
)

nutritionist_planner_task = Task(
    description="""Generate a detailed daily meal plan for the client, aligned with Brazilian cuisine preferences. Include educational resources and guidance on healthy eating habits.
    Therefore, use the client profile to give suggestions about what the client can do to achieve the goals and also what he has to do better and start doing.
    The plan should be practical, sustainable, and tailored to the client's goals.""",
    expected_output="A fully developed meal planner in markdown format and to save as pdf format, structured as follows:\n"
    "1. ## Overview;\n"
    "2. ## Daily Plan with 3 Options for Each Meal (Breakfast, Lunch, Afternoon Snacks, Dinner, Supper):\n"
    "2.1 - **### Breakfast:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
    "2.2 - **### Lunch:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>, including salads and vegetables\n"
    "2.3 - **### Afternoon Snacks:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
    "2.4 - **### Dinner:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>, including salads and vegetables\n"
    "2.5 - **### Supper:** - Option 1: <answer>\n - Option 2: <answer>\n - Option 3: <answer>\n"
    "3. ## Educational Resources and Guidance (including water intake in liters);\n"
    "4. ## Activities, Tips, and Exercises Plan for the Gym;\n"
    "e.g. \n"
    "4.1 - ### Monday: Back and biceps day and 30min cardio.,\n"
    "4.2 - ### Tuesday: Chest day and abs exercises...etc\n"
    "4.3 - ...etc..."
    "5. ## Cooking Tips and Techniques for the Meal Plan;\n"
    "6. ## Additional Information Based on the Client's Profile;\n",
    agent=nutritionist_agent,
    context=[profile_analysis_task, diet_recommendation_task, educational_support_task],
    output_file="outputs/caiobarros_planner.pdf",
    create_directory=True,
)

In [4]:
%%time

# Assemble the Crew
crew = Crew(
    agents=[
        profile_analysis_agent,
        diet_recommendation_agent,
        educational_support_agent,
        nutritionist_agent,
    ],
    tasks=[
        profile_analysis_task,
        diet_recommendation_task,
        educational_support_task,
        nutritionist_planner_task,
    ],
)

# Kickoff the Crew
result = crew.kickoff(
    inputs={
        "user_profile": {
            "name": "Caio",
            "age": "26",
            "height": "1.80m",
            "gender": "Male",
            "main_health_goals": "Weight loss, reduce fat percentage, and focus on reducing abdominal fat, especially on the flanks",
            "medical_conditions_or_food_allergies": "No, I don't",
            "current_diet": {
                "morning": "Wholemeal bread with two eggs and a cup of sugar-free coffee.",
                "lunch": "100g of rice, 100g of shredded chicken, and 100g of beans.",
                "afternoon_snack": "180g of low-fat yogurt, 30g of whey protein, and a fruit (grape, apple, or guava).",
                "dinner": "100g of rice, 100g of shredded chicken, and 100g of beans.",
                "supper": "Smoothie of 100g frozen strawberries with 30g powdered milk and 30g whey protein.",
            },
            "frequency_of_eating_out_or_processed_foods": "Now and then I eat sliced turkey breast.",
            "supplements_or_medications": "Whey protein, omega 3, caffeine (200mg), and creatine.",
            "physical_activity_level": "I go to the gym 5 times a week and push high lifts at a median level.",
            "feelings_after_eating": "Normal, usually not satisfied so much.",
            "water_intake": "I need to perform better to this. I would say 2 liters per day.",
            "sleep_quality_and_duration": "Among 6h and 8 hours per day.",
            "dietary_preferences_or_restrictions": "No, I don't have any preferences.",
            "alcohol_or_caffeinated_beverages_consumption": "I like to drink coffee and I also use caffeine supplement.",
            "stress_levels_and_management": "Median.",
            "cultural_or_religious_dietary_practices": "No, I don't have.",
        }
    }
)

print("############")
print(result)

############
**Comprehensive Nutrition and Lifestyle Plan for Caio**

**Overview**

Congratulations on taking the first step towards a healthier lifestyle! This comprehensive nutrition and lifestyle plan is designed to provide a balanced and healthy diet that meets your health and fitness objectives. The plan includes a detailed daily meal plan, educational resources, and guidance on healthy eating habits and nutritional guidelines.

**Daily Meal Plan**

The daily meal plan is divided into three main meals and two snacks, with a focus on incorporating salads and vegetables in lunch and dinner.

**Breakfast**

* Monday: Oatmeal with banana, almond butter, and a sprinkle of cinnamon
* Tuesday: Scrambled eggs with spinach, mushrooms, and whole wheat toast
* Wednesday: Greek yogurt with berries, honey, and chopped nuts
* Thursday: Avocado toast on whole grain bread with a fried egg
* Friday: Smoothie bowl with protein powder, almond milk, banana, spinach, and almond butter topping

**Lunch